In [21]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.insert(1, oj(sys.path[0], '..', 'vision_fit'))  # insert parent path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import math
# plt.style.use('dark_background')
from mog_fit import data
from collections import OrderedDict
from sklearn import preprocessing

import torch
from torch.autograd import Variable
from mog_analyze import viz
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

from vision_fit import data
import viz_weights
from process_results import process_results

%matplotlib inline
%load_ext autoreload
%autoreload 2

import style
cb = '#008fd5'
cr = '#fc4f30'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
style.set_style()

# load results from a directory

In [ ]:
# depending on how much is saved, this may take a while
out_dir = '/scratch/users/vision/yu_dl/raaz.rsk/track_acts/resweep_full_new'
out_dir2 = '/scratch/users/vision/yu_dl/raaz.rsk/track_acts/resweep_full_new2'
fnames = sorted([oj(out_dir, fname) for fname in os.listdir(out_dir) \
                 if not fname.startswith('weights') \
                 and not fname.startswith('idx')]) + \
         sorted([oj(out_dir2, fname) for fname in os.listdir(out_dir2) \
                 if not fname.startswith('weights') \
                 and not fname.startswith('idx')])    

results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in tqdm(fnames) 
                if not fname.startswith('weights') and not fname.startswith('idx')]
results = pd.concat(results_list, axis=1).T.infer_objects()
print(results.shape)

**add cols**

In [87]:
# adds these vec keys: fc0_fro, fc1_fro, fc0_stab_rank, fc1_stab_rank, act0_stab_rank, act1_stab_rank, corr0, corr1
# adds these scalar keys: max_train_acc, max_test_acc, _final of all the above
print('before', results.shape)
its, ts, results = process_results(results)
print('after', results.shape)

before (3034, 54)
after (2856, 84)


# plot max corrs
*note some things disappear in below plot as a result of nans*

In [ ]:
r = results
r = r[r.dset == 'mnist']
r = r[r.hidden_size == 128]
r = r[r.max_test_acc >= 0.97]
hue = 'optimizer'

R, C = 1, 1
plt.figure(figsize=(14, 14), facecolor='w', dpi=100)
plt.subplot(R, C, 1)
plt.title('mnist')

x = np.array(r.max_test_acc)
y = np.array(r.corr0_final)
pids = np.array([pid[:5] for pid in r.pid])
ax = sns.scatterplot(x, y, hue=r[hue])

for i, pid in enumerate(pids):
    ax.annotate(pid, (x[i], y[i]))

plt.show()

# pick examples

In [108]:
pre_a = '24421'
pre_s = '40043'

for pid in r.pid:
    if pid.startswith(pre_a):
        print('a', pid)
    if pid.startswith(pre_s):
        print('s', pid)

# good for 128
# pid_a = '24421020350011741116'
# pid_s = '40043048138378577087'

# good for 512
pid_a = '07665771801545027002'
pid_s = '24656068102223723425'

s 40043048138378577087
a 24421020350011741116


In [109]:
# depending on how much is saved, this may take a while
out_dir = '/scratch/users/vision/yu_dl/raaz.rsk/track_acts/resweep_full_new'
out_dir2 = '/scratch/users/vision/yu_dl/raaz.rsk/track_acts/resweep_full_new2'
fnames = sorted([oj(out_dir, fname) for fname in os.listdir(out_dir) \
                 and (pid_a in fname or pid_s in fname)]) + \
         sorted([oj(out_dir2, fname) for fname in os.listdir(out_dir2) \
                 and (pid_a in fname or pid_s in fname)])

fnames_w = [fname for fname in fnames if fname.startswith('weights')]

results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in fnames_w]
rw = pd.concat(results_list, axis=1).T.infer_objects()

rw_sgd = rw[rw.optimizer == 'sgd'].reset_index().iloc[0]
rw_adam = rw[rw.optimizer == 'adam'].reset_index().iloc[0]

print(rw.keys())
print(rw.hidden_size)

Index(['batch_size', 'calc_activations', 'dset', 'first_layer_lr_mult',
       'freeze', 'hidden_size', 'its', 'lr', 'lr_step', 'lr_ticks',
       'num_iters', 'num_iters_small', 'num_layers', 'num_points', 'optimizer',
       'out_dir', 'pid', 'save_acts_and_reduce', 'save_all_freq',
       'save_all_weights_freq', 'saves_per_iter', 'saves_per_iter_end', 'seed',
       'shuffle_labels', 'use_conv', 'use_conv_special', 'weights',
       'weights_first10'],
      dtype='object')
0    512
1    512
Name: hidden_size, dtype: int64


# remake weight plots with hist
*sort the weights by highest mem to lowest mem*

In [ ]:
from matplotlib.ticker import NullLocator

for run in [rw_sgd, rw_adam]:
    run['num_layer'] = int(run['num_layers'])
    run['hidden_size'] = int(run['hidden_size'])
    epoch = int(max(ts))
    lay = 'fc.0.weight'

    # keys are epochs, vals are dicts of all weights
    weights_dict_dict = run['weights']

    # keys are layers, vals are weight values
#     print(weights_dict_dict.keys())
    weights_dict = weights_dict_dict[epoch]
#     print(weights_dict.keys())

    # matrix of weights (output x input)
    w = weights_dict[lay]
    viz_weights.plot_weights(w, dset='mnist', C=18, dpi=120)
                
    plt.gca().set_axis_off()
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.margins(0, 0)

    plt.savefig(run.optimizer + '_' + str(run.hidden_size) + '_' + 'w0.pdf', bbox_inches = 'tight', pad_inches = 0)
    plt.show()

In [ ]:
X = X_train.cpu().numpy().reshape(X_train.shape[0], -1)
W = model.state_dict()['fc.0.weight'].cpu().numpy()
max_corr = max_corr_input(X, W)
print('mean max corr', np.mean(max_corr))
plt.hist(max_corr)
plt.axvline(np.mean(max_corr), color='red')
plt.show()

# make alterations

In [ ]:
import calc_max_corr

# load model
model = data.get_model(run)

train_loader, test_loader = data.get_data_loaders(run)
X_train, Y_train, X_test, Y_test = calc_max_corr.process_loaders(train_loader, test_loader)

# load in weights
weights_dict_tensors = {k: torch.Tensor(v) for k, v in weights_dict.items()}
model.load_state_dict(weights_dict_tensors)
model = model.cuda()

In [ ]:
X = X_train.cpu().numpy().reshape(X_train.shape[0], -1)[:1000]
Y = Y_train[:1000]
X = X[Y.argsort()] # sort X by class
W1 = model.state_dict()['fc.0.weight'].cpu().numpy()
Z = X @ W1.T

# Z[:, -50:] = np.tile(Y[Y.argsort()], 50).reshape(-1, 50)

In [ ]:
preds = model(Variable(X_train)).data.cpu().numpy().argmax(axis=1)
accs = preds==Y_train
print('mean acc', np.mean(accs))